## Описание проекта
Интернет-магазин запускает новый сервис, чтобы пользователи могли редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.


## Загружу и подготовлю данные.

Установлю отсутствующие в окружении библиотеки:

In [ ]:
!pip install catboost

In [ ]:
!pip3 install torch torchvision

In [ ]:
!pip install transformers

In [ ]:
!pip install tensorflow

In [ ]:
!pip install --upgrade gensim

In [52]:
# переустановка и апгрейд numpy, если есть проблемы с использованием Gensim
#!pip uninstall numpy
#!pip install numpy
#!pip install numpy --upgrade

Импортирую библиотеки:

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

In [2]:
import torch
import transformers

In [3]:
# для эмбендингов
import tensorflow
from transformers import TFBertModel

In [4]:
# для лемматизации через WordNetLemmatizer
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download() # загрузка пакетов через всплывающее окно

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [5]:
# для SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

### Изучу общую информацио о данных

In [6]:
df = pd.read_csv('toxic_comments.csv')

In [7]:
df.shape

(159571, 2)

In [8]:
df.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [10]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Данные полные, соотношение классов нормальное. Для дальнейшей работы с ними, требуется предобработка.

### Проведу лемматизацию:

In [11]:
corpus = df['text'].values.astype('U')

#### Напишу свои функции для лемматизации

Сохраню 5 строк для тестирования, чтобы не ждать долго:

In [12]:
df_exp = df.head().copy()
df_exp

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [13]:
def lemmatize(text):
    m = Mystem()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
    return lemm_text.lower()

<div class="alert alert-block alert-info">
<b>Note: </b> Оказывается, этот лемматизатор работает только с русским языком, так что его нет смысла применять.
</div>

In [1]:
def clear_text(text): # функция для очистки текста
    return " ".join(re.sub(r'[^a-zA-Z]', ' ', text).split())

Проверю работу функций на первом элементе корпуса:

In [15]:
print("Исходный текст:", corpus[0])
print("Очищенный и лемматизированный текст в нижнем регистре:", lemmatize(clear_text(corpus[0])))

Исходный текст: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Очищенный и лемматизированный текст в нижнем регистре: explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now



Уже на 1 предложении видно, что лемматизация происходит медленно. Попробую циклом пройтись по первым 5 элементами корпуса:

In [16]:
%%time
lemm_text_list = []
for i in list(df_exp['text'].head()):
    lemm_text_list.append(lemmatize(clear_text(i)))
lemm_text_list

CPU times: user 7.46 ms, sys: 32.2 ms, total: 39.6 ms
Wall time: 23 s


['explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now\n',
 'd aww he matches this background colour i m seemingly stuck with thanks talk january utc\n',
 'hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info\n',
 'more i can t make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know th

Другой метод пройти по столбцу:

In [17]:
corpus_exp = df_exp['text'].values.astype('U')

Лемматизирую, но прежде очищу с помощью созданной ранее фукции:

In [18]:
%%time
df_exp['lemm_text'] = [lemmatize(clear_text(i)) for i in corpus_exp]

CPU times: user 8.09 ms, sys: 30.9 ms, total: 39 ms
Wall time: 7.13 s


In [19]:
df_exp

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


Лемматизация через цикл работает очень медленно, не буду использовать этот метод.  
Думаю, стоит использовать модели с встроенной лемматизацией и токенизацией или иной метод лемматизации, чтобы попробовать Логистическую регрессию.

#### Попробую лемматизацию WordNetLemmatizer

In [20]:
stopword = stopwords.words('english')
snowball_stemmer = SnowballStemmer('english')

In [21]:
corpus_exp = df_exp['text'].values.astype('U')

Инициализирую Wordnet Lemmatizer:

In [22]:
lemmatizer = WordNetLemmatizer()

In [23]:
# проверяю работу Wordnet Lemmatizer на примере:
print(lemmatizer.lemmatize("Tests").lower())

tests


In [24]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z]', ' ', text).split())

In [25]:
def lemm_text(text):
    #lowerized = lemmatizer.lemmatize(text) # лемматизирую
    word_tokens = nltk.word_tokenize(text) # разбиваю текст на токены-слова
    removing_stopwords = [word for word in word_tokens if word not in stopword] # удаляю стоп-слова
    wordnet_lemmatizer = WordNetLemmatizer() 
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in removing_stopwords] # лемматизирую
    return clear_text(text).lower()

In [26]:
%%time
df['lemm_text'] = df['text'].apply(lemm_text)

CPU times: user 3min 9s, sys: 1.37 s, total: 3min 10s
Wall time: 3min 25s


Лемматизация работает.

### Разобью датасет на выборки

In [27]:
train, test = train_test_split(df, test_size=0.2,  random_state=12345)
train.shape, test.shape

((127656, 3), (31915, 3))

In [28]:
train, valid = train_test_split(
    train, test_size=0.25,  random_state=12345)
train.shape, valid.shape

((95742, 3), (31914, 3))

## Обучу разные модели.

### Catboost

In [29]:
model = CatBoostClassifier(verbose=100,
                           iterations=1500,
                           learning_rate=0.7,
                           early_stopping_rounds=200,
                           eval_metric='F1',
                           auto_class_weights='Balanced',
                           random_seed=12345,
                           use_best_model=True,
                           depth=8
                           )

In [30]:
%%time
model.fit(train[['text']], train[['toxic']],
          eval_set=(valid[['text']], valid[['toxic']]),
          text_features=['text'])

0:	learn: 0.8122141	test: 0.8370509	best: 0.8370509 (0)	total: 1.19s	remaining: 29m 39s
100:	learn: 0.9227164	test: 0.8648860	best: 0.8744499 (20)	total: 57.9s	remaining: 13m 22s
200:	learn: 0.9533163	test: 0.8533223	best: 0.8744499 (20)	total: 1m 55s	remaining: 12m 24s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.874449926
bestIteration = 20

Shrink model to first 21 iterations.
CPU times: user 6min 8s, sys: 57 s, total: 7min 5s
Wall time: 2min 18s


In [31]:
pred_catboost = model.predict(test[['text']])
print(classification_report(test['toxic'], pred_catboost))

              precision    recall  f1-score   support

           0       0.98      0.89      0.94     28676
           1       0.47      0.84      0.61      3239

    accuracy                           0.89     31915
   macro avg       0.73      0.87      0.77     31915
weighted avg       0.93      0.89      0.90     31915



In [32]:
f1_score(test['toxic'], pred_catboost)

0.6063971568191916

Маленькое значение метрики ;(

### LogisticRegression

In [33]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elizaveta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
tfidf = TfidfVectorizer(stop_words=stopwords)

In [35]:
%%time
corpus_train = train['lemm_text'].values.astype('U')
tfidf_train = tfidf.fit_transform(corpus_train)

CPU times: user 7.94 s, sys: 2.03 s, total: 9.97 s
Wall time: 10.5 s


In [36]:
model = LogisticRegression(class_weight='balanced', max_iter=200, random_state=12345, verbose=50)
model.fit(tfidf_train, train['toxic'])

[LibLinear]

/Users/elizaveta/opt/anaconda3/envs/praktikum/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=200, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=50,
                   warm_start=False)

In [37]:
corpus_valid = valid['lemm_text'].values.astype('U')
tfidf_valid = tfidf.transform(corpus_valid)

In [38]:
predictions = model.predict(tfidf_valid)

In [39]:
print('Метрика F1 для Линейной регрессии:', f1_score(valid['toxic'], predictions))

Метрика F1 для Линейной регрессии: 0.7527863569222506


На Линейной регрессии получил приемлемую метрику F1.

### SGDClassifier

In [40]:
sgdc_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(
        loss='hinge', # в кач-ве функции потерь использую Support Vector Machine
        penalty='l2',
        alpha=1e-3,
        random_state=12345,
        max_iter=5,
        tol=None,
        verbose=10)),
])

In [41]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

In [42]:
gs_clf = GridSearchCV(sgdc_clf, parameters, n_jobs=-1, verbose=1)

In [43]:
%%time
gs_clf.fit(train['lemm_text'], train['toxic'])

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/Users/elizaveta/opt/anaconda3/envs/praktikum/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.8min finished


-- Epoch 1
Norm: 5.75, NNZs: 38262, Bias: -0.996527, T: 95742, Avg. loss: 0.167703
Total training time: 0.05 seconds.
-- Epoch 2
Norm: 5.73, NNZs: 41808, Bias: -0.994547, T: 191484, Avg. loss: 0.165991
Total training time: 0.11 seconds.
-- Epoch 3
Norm: 5.71, NNZs: 42674, Bias: -0.994177, T: 287226, Avg. loss: 0.165699
Total training time: 0.18 seconds.
-- Epoch 4
Norm: 5.71, NNZs: 43070, Bias: -0.993695, T: 382968, Avg. loss: 0.165656
Total training time: 0.25 seconds.
-- Epoch 5
Norm: 5.70, NNZs: 43257, Bias: -0.993464, T: 478710, Avg. loss: 0.165566
Total training time: 0.33 seconds.
CPU times: user 9.22 s, sys: 2.05 s, total: 11.3 s
Wall time: 2min 52s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [44]:
predictions = gs_clf.predict(valid['lemm_text'])

In [45]:
gs_clf.best_score_
gs_clf.best_params_

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

In [46]:
print('Метрика F1 для SGDClassifier:', f1_score(valid['toxic'], predictions))

Метрика F1 для SGDClassifier: 0.23363491218733373


Метрика очень плохая, GridSearch не нашёл подходящий параметр или что-то нет так в подготовке данных.

## Сделаю выводы.

Попробовал разные модели:
- CatBoost: получил низкое значение метрики F1 (0.627).
- LogisticRegression: если использовать быструю лемматизацию (WordNetLemmatizer), то модель работает довольно быстро и выдаёт хорошую метрику F1 (0.752).
- SGDClassifier с GridSearch: из-за перебора модель работает долго и на выходе показывает неприлично слабую метрику F1 (0.224). Тут нужно экспериментировать с параметрами или что-то исправить во входных данных для обучения, модель должна быть на уровне с логистической регрессией.  

Самой эффектиной в моей работе оказалась Линейная регрессия с качественной предподготовкой данных.